In [ ]:
import requests
import json
from datetime import datetime
import pandas as pd
import os
import time

def formatar_cnpj(cnpj):
    # Remove qualquer caractere que não seja número
    cnpj_limpo = ''.join(filter(str.isdigit, cnpj))
    if len(cnpj_limpo) != 14:
        return None
    return cnpj_limpo

def buscar_cnpj(cnpj):
    url = f"https://www.receitaws.com.br/v1/cnpj/{cnpj}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

def gerar_nome_arquivo():
    data_atual = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    versao = 1
    while True:
        nome_arquivo = f"resultados_processamento_v{versao}_{data_atual}.xlsx"
        if not os.path.exists(nome_arquivo):
            return nome_arquivo
        versao += 1

def processar_arquivo_excel(caminho_arquivo):
    try:
        df = pd.read_excel(caminho_arquivo, usecols=[0])  # Lê apenas a primeira coluna
        df.columns = ['CNPJ']  # Renomeia a coluna para 'CNPJ'
        
        resultados = []
        total = len(df)
        print(f"Total de CNPJs a serem processados: {total}")
        
        for index, row in df.iterrows():
            cnpj_original = str(row['CNPJ'])
            cnpj_formatado = formatar_cnpj(cnpj_original)
            
            if cnpj_formatado:
                print(f"Processando CNPJ {index+1}/{total}: {cnpj_formatado}")
                dados = buscar_cnpj(cnpj_formatado)
                
                if dados:
                    resultados.append({
                        'CNPJ': cnpj_original,
                        'Nome': dados.get('nome', ''),
                        'Fantasia': dados.get('fantasia', ''),
                        'Logradouro': dados.get('logradouro', ''),
                        'Número': dados.get('numero', ''),
                        'Complemento': dados.get('complemento', ''),
                        'Município': dados.get('municipio', ''),
                        'UF': dados.get('uf', ''),
                        'CEP': dados.get('cep', ''),
                        'Telefone': dados.get('telefone', ''),
                        'Email': dados.get('email', ''),
                        'Situação': dados.get('situacao', ''),
                        'Data da Situação': dados.get('data_situacao', ''),
                        'CNAE Principal': dados.get('atividade_principal', [{}])[0].get('text', ''),
                        'Capital Social': dados.get('capital_social', ''),
                        'Porte': dados.get('porte', ''),
                        'Natureza Jurídica': dados.get('natureza_juridica', ''),
                        'Status': 'Sucesso'
                    })
                else:
                    resultados.append({'CNPJ': cnpj_original, 'Status': 'Falha'})
            else:
                print(f"CNPJ inválido: {cnpj_original}")
                resultados.append({'CNPJ': cnpj_original, 'Status': 'CNPJ Inválido'})

            # Pausa para evitar sobrecarga em solicitações consecutivas
            time.sleep(0.2)
        
        # Cria um DataFrame com os resultados e salva em um arquivo Excel com versão e data
        df_resultados = pd.DataFrame(resultados)
        nome_arquivo_resultados = gerar_nome_arquivo()
        df_resultados.to_excel(nome_arquivo_resultados, index=False)
        print(f"Resumo do processamento salvo em: {nome_arquivo_resultados}")
        
    except Exception as e:
        print(f"Erro ao processar o arquivo: {str(e)}")

def main():
    while True:
        escolha = input("Escolha uma opção:\n1 - Processar CNPJ único\n2 - Processar arquivo Excel\n3 - Sair\nSua escolha: ")
        
        if escolha == '1':
            cnpj = input("Digite o CNPJ: ")
            cnpj_formatado = formatar_cnpj(cnpj)
            
            if cnpj_formatado:
                print("Buscando informações...")
                dados = buscar_cnpj(cnpj_formatado)
                
                if dados:
                    print("\nInformações da empresa:")
                    print(f"CNPJ: {dados.get('cnpj', '')}")
                    print(f"Nome: {dados.get('nome', '')}")
                    print(f"Fantasia: {dados.get('fantasia', '')}")
                    print(f"Logradouro: {dados.get('logradouro', '')}, {dados.get('numero', '')} {dados.get('complemento', '')}")
                    print(f"Município: {dados.get('municipio', '')}")
                    print(f"UF: {dados.get('uf', '')}")
                    print(f"CEP: {dados.get('cep', '')}")
                    print(f"Telefone: {dados.get('telefone', '')}")
                    print(f"Email: {dados.get('email', '')}")
                    print(f"Situação: {dados.get('situacao', '')}")
                    print(f"Data da Situação: {dados.get('data_situacao', '')}")
                    print(f"CNAE Principal: {dados.get('atividade_principal', [{}])[0].get('text', '')}")
                    print(f"Capital Social: {dados.get('capital_social', '')}")
                    print(f"Porte: {dados.get('porte', '')}")
                    print(f"Natureza Jurídica: {dados.get('natureza_juridica', '')}")
                else:
                    print("Não foi possível obter as informações. Verifique o CNPJ e tente novamente.")
            else:
                print("CNPJ inválido. Por favor, digite um CNPJ válido.")
        
        elif escolha == '2':
            caminho_arquivo = input("Digite o caminho completo do arquivo Excel: ")
            if os.path.exists(caminho_arquivo):
                processar_arquivo_excel(caminho_arquivo)
            else:
                print("Arquivo não encontrado. Verifique o caminho e tente novamente.")
        
        elif escolha == '3':
            print("Encerrando o programa...")
            break
        
        else:
            print("Opção inválida. Por favor, escolha 1, 2 ou 3.")

if __name__ == "__main__":
    main()

Escolha uma opção:
1 - Processar CNPJ único
2 - Processar arquivo Excel
3 - Sair
Sua escolha:  1
Digite o CNPJ:  71673990000177


Buscando informações...

Informações da empresa:
CNPJ: 71.673.990/0001-77
Nome: NATURA COSMETICOS S/A
Fantasia: NATURA
Logradouro: AV ALEXANDRE COLARES, 1188 
Município: SAO PAULO
UF: SP
CEP: 05.106-000
Telefone: (11) 4389-7317
Email: fiscal@natura.net
Situação: ATIVA
Data da Situação: 03/11/2005
CNAE Principal: Comércio atacadista de cosméticos e produtos de perfumaria
Capital Social: 2000000000.00
Porte: DEMAIS
Natureza Jurídica: 204-6 - Sociedade Anônima Aberta


Escolha uma opção:
1 - Processar CNPJ único
2 - Processar arquivo Excel
3 - Sair
Sua escolha:  2
Digite o caminho completo do arquivo Excel:  C:\Users\arthur.santana\Desktop\FORNECEDORES_TESTE_SANEAMENTO_CNPJS.xlsx


Total de CNPJs a serem processados: 5347
Processando CNPJ 1/5347: 16670085008997
Processando CNPJ 2/5347: 00512777000135
Processando CNPJ 3/5347: 78943321000117
Processando CNPJ 4/5347: 29926961000103
Processando CNPJ 5/5347: 02012862000160
Processando CNPJ 6/5347: 33136896000190
Processando CNPJ 7/5347: 02288055000174
Processando CNPJ 8/5347: 81071623000238
Processando CNPJ 9/5347: 02238702000214
Processando CNPJ 10/5347: 02139652000218
Processando CNPJ 11/5347: 50710730000154
Processando CNPJ 12/5347: 33896614000152
Processando CNPJ 13/5347: 09344391000217
Processando CNPJ 14/5347: 80732928000108
Processando CNPJ 15/5347: 36212637000199
Processando CNPJ 16/5347: 37069853000190
Processando CNPJ 17/5347: 00124587000140
Processando CNPJ 18/5347: 33363415001122
Processando CNPJ 19/5347: 00146461000177
Processando CNPJ 20/5347: 33643420000226
Processando CNPJ 21/5347: 33937681000178
Processando CNPJ 22/5347: 02088938000130
Processando CNPJ 23/5347: 01943121000300
Processando CNPJ 24/5347: